# AMAL - TP 5  

## 循环神经网络：序列到序列 (seq2seq)

### 简介（简短，参见课程）

在这个关于RNN的第三个也是最后一个实践课（TP）中，我们将研究两类“序列到序列”（Seq2Seq）任务：

- 对序列的每个元素进行标注（输入和输出的元素数量相同）
- 从一个隐藏状态生成一个序列。这个隐藏状态表示输入数据，并根据任务的不同有所变化：可以是句子的表示（例如翻译/问答任务）、图像的表示（例如图像描述）等

与之前的实践课相比，这次有三大新内容：
1. 使用由torch定义的RNN，它们接受PackedSequence作为输入，并返回类似的结构（也可以与标准批处理一起使用）。
2. 使用可变大小的编码（wordpiece）。
3. 使用课程学习（curriculum learning）进行训练。

### 1. 标注

在本练习中，我们将关注句法分析任务（词性标注，Part-Of-Speech），该任务旨在为每个词语分配一个词性或语法类别。我们将使用GSD数据集（参见代码框架获取数据集的下载方式）。每个样本都是一个已经分词的句子，我们将关注词语的原形（form）和词性标签（upostag）。

你将获得以下代码：
- `Vocabulary`类：用于管理词汇表（即词语与其索引的对应关系）；它将用于为词语和词性标签建立索引。
- `TaggingDataset`类：用于构建数据集，其中每个项目都是一个（词语，标签）的组合，词语是句子中的词，而标签是与之对应的类别。
- DataLoader的`collate_fn`函数：它使用`pad_sequence`函数来使所有序列的长度一致（详见下文）。

在之前的实验中提到的填充（padding），可以通过Pytorch的`pad_sequence`函数来完成。`pad_sequence`允许我们将序列补齐到相同长度。在计算损失时，损失函数的`ignore_index`参数可以指定填充值的索引，以忽略填充部分的影响（类似于上次课程中的掩码）。然而，填充序列可能带来很高的计算成本，因为许多无效操作将被执行（包括网络的输入和损失的计算过程中）。

Pytorch 提供了一种解决这种额外开销的方法，称为“打包”序列（packed sequences）：序列会被压平并对齐到相同的维度，序列的实际长度会被同时存储，这样只执行必要的操作。可以通过`pack_sequence`函数将一个张量列表转换为`PackedSequence`，这是一个包含两个部分的元组：一部分是打包后的batch，另一部分是batch中每个元素的长度。对于已经填充好的batch，可以使用`pack_padded_sequence`来获得这个压平的张量。而反向操作则是`pad_packed_sequence`，它可以将压平后的张量转换回填充的张量。

**问题 1**

实现一个用于标注的seq2seq模型，并使用Pytorch的LSTM模块和序列填充（作为附加内容，可以尝试实现序列打包packing）。为了处理在测试时可能出现的未知词（OOV，out of Vocabulary）问题，在训练过程中可以随机将样本中的一些词替换为OOV词（使用一个特殊的token - [OOV]）。实现该功能并比较结果。

为了可视化结果，对于给定的输入句子，显示与之对应的标注序列。

a. 与之前的实验不同，我们现在处理的是词元（tokens），而不是字符。在测试时，很可能会遇到一些只在测试中出现的词，这些词被称为OOV（超出词汇表的词）。

### 2.翻译

在翻译任务中，我们将使用两个 RNNs：
- 一个编码器，用于读取要翻译的序列后生成一个隐藏状态；
- 一个解码器，从隐藏状态开始生成翻译后的句子。

除了 EOS（序列结束）标记外，您还需要一个特殊的 SOS（序列开始）标记，它将作为解码器的第一个输入标记（加上隐藏状态），从该标记开始生成翻译句子。

训练解码器有两种方式：
- **约束模式**（或称为“教师强制”模式），其中将目标句子传递给解码器，并在每个时间步中使用目标句子中的一个词作为参考。这种生成方式受到指导，能够精确地校正每个生成的潜在状态。
- **非约束模式**，在这种模式下，目标句子不会在生成过程中被考虑。在迭代生成翻译时，每个时间步的输入是由前一步的潜在状态解码出的概率最大词（或从该分布中随机采样的词）。这种模式模拟推断过程，先生成整个句子，再对生成的句子进行修正。

非约束模式比约束模式更困难：预测一个词时的错误会极大干扰接下来的生成，导致后续序列的反向传播效果不佳。然而，非约束模式能够更好地泛化，避免死记硬背。直观上，应该先使用约束模式训练解码器以进行良好的初始化，然后逐渐转向非约束模式。这种方法称为**课程学习**（Curriculum Learning）。

**问题 2**

在 `tp5-traduction.py` 中，实现编码器-解码器。对于两者，使用 GRU 和以下架构：
- **编码器**：首先对源词汇进行嵌入（embedding），然后使用 GRU。
- **解码器**：首先对目标词汇进行嵌入，然后使用 GRU，接着是一个线性网络用于解码潜在状态（最后使用 softmax）。

在解码器中，您需要一个方法 `generate(hidden, lenseq=None)`，该方法根据给定的隐藏状态 `hidden`（以及输入的 SOS 令牌）生成一个序列，直到达到 `lenseq` 长度或生成 EOS 令牌为止。

实现学习循环，使用一种简单的课程学习策略，该策略是为每个小批量在约束模式和非约束模式之间均匀随机抽取。您可以在生成时传递目标句子的期望长度，尤其是在非约束模式下。

训练模型时，保留一个测试集，以确保您不会过拟合或欠拟合。使用上一个任务中的生成方法来可视化所提议的翻译。

a. 在原始文章中，选择两种模式是基于对约束模式的递减概率，并且这个选择是在每个时间步进行的，而不是针对整个小批量。作为附加任务，您可以改进该策略并比较结果。

### 3 文本的预处理和有效分割

文本的预处理依赖于一个分割步骤，在这个步骤中，文本被切割成语言单位。很长一段时间以来，选择的分割级别是单词（即被空格包围的字母数字字符串）；近年来，随着新的神经网络模型的出现，已经重新探索了一些替代方案。

目前最有效的分割方法之一是可变 n-gram（子词单元）分割，这种方法在 2016 年通过字节对编码（Byte-Pair Encoding，BPE）而受到广泛关注。这些分割方法的优点在于它们具有固定大小的词汇表，可以尽可能全面地覆盖数据集，并且能够避免未知词汇的问题。

例如，“You shoulda got David Carr of Third Day to do it” 将被分割为 "_You", "_should", "a", "_got", "_D", "av", "id", "_C", "ar", "r", "_of", "_Th", "ir", "d", "_Day", "_to", "_do", "_it"，其中频繁的序列（例如 You, should）被直接提取，而不太频繁的序列（例如 David, Carr）则被分割成多个部分。

您可以使用 `sentencepiece` 库（如果您不在大学的机器上工作，请记得更新 Python 模块）。未知标记 `<unk>`、开始标记 BOS (`<s>`) 和结束标记 EOS (`</s>`) 是预定义的，但您可以通过 `user_defined_symbols` 添加其他标记：

In [ ]:
import sentencepiece as spm

spm.SentencePieceTrainer.train(
    input='FILEPATH',
    model_prefix='MODEL_OUTPUT_PATH',
    vocab_size=1000,
    user_defined_symbols=[]
)

# 用于分割句子的指令如下：
import sentencepiece as spm

s = spm.SentencePieceProcessor(model_file='MODELPATH.model')

# 返回标记的标识符
ids = s.encode('New York', out_type=int)

# 返回对应的字符串
s.decode(ids)

# 返回标记
s.encode('New York', out_type=str)


### 问题 3
创建用于数据集的分割模型，然后更改数据集以使用可变分割。一旦分割完成，就着手解决翻译模型的学习问题。您可以比较使用分割和不使用分割的结果。